## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)
print(gmaps)

<module 'gmaps' from 'C:\\Users\\lself\\anaconda3\\envs\\PythonData\\lib\\site-packages\\gmaps\\__init__.py'>


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,49.69,87,2,12.06,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,73.20,70,70,20.15,broken clouds
2,2,Busselton,AU,-33.6500,115.3333,49.69,87,2,12.06,clear sky
3,3,Port Keats,AU,-14.2500,129.5500,74.79,44,0,10.92,clear sky
4,4,Bambous Virieux,MU,-20.3428,57.7575,68.22,78,94,16.71,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Minimum Temperature: '))
max_temp = float(input('Maximum Temperature: '))

Minimum Temperature: 68
Maximum Temperature: 72


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_match_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
temp_match_cities_df.isna().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_match_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,
4,Bambous Virieux,MU,68.22,light rain,-20.3428,57.7575,
25,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,
30,Grand Gaube,MU,68.38,broken clouds,-20.0064,57.6608,
39,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,
40,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,
50,Coruripe,BR,73.67,few clouds,-10.1256,-36.1756,
55,Omboué,GA,72.57,overcast clouds,-1.5746,9.2618,
56,Mahébourg,MU,68.25,light rain,-20.4081,57.7000,
65,Carballo,ES,69.22,scattered clouds,43.2130,-8.6910,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print(f"No hotel found near {params['location']}")
print("Complete.")

No hotel found near 13.2954,19.6966
No hotel found near 11.6667,-9.5
No hotel found near -6.4667,16.8167
No hotel found near -21.35,165.15
No hotel found near -10.4,40.3333
No hotel found near 4.7413,22.8184
No hotel found near 51.4185,36.7499
No hotel found near -20.2333,164.0167
No hotel found near -6.4667,16.8167
Complete.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,People ThankYou
4,Bambous Virieux,MU,68.22,light rain,-20.3428,57.7575,Casa Tia Villa
25,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,People ThankYou
30,Grand Gaube,MU,68.38,broken clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
39,Rikitea,PF,73.20,broken clouds,-23.1203,-134.9692,People ThankYou


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'gmaps' is not defined